# **Quantifier Depth(QD) in FOL with BOS**

## **1. Introduction**

This document is written in **RASP** (Restricted Access Sequence Processing), a domain-specific language introduced in *Thinking Like Transformers (Weiss et al., 2021)*, which approximates the self-attention mechanism in the Transformer model. 

The algorithms in this document focus on computing Quantifier Depth (QD) in First-order Logic (FOL) expressions with the **beginning-of-sequence token (§)**. Syntactically, QD represents the maximum nesting level of quantifiers (e.g., ∀, ∃) in a formula, and is widely used to measure structural logical complexity.

A total of **three Standard types** are covered, each corresponding to a distinct logical structure with varying syntactic depth. The types are ordered with increasing complexity, based on their nesting behavior and compositional logic.

For each **Type**:

- **A RASP algorithm** implements QD computation for instances conforming to that grammar;

- **Several Test Cases** includes "(x/x)" annotated examples demonstrating both the algorithm’s expected behavior and its generalization. The final block in this section follows this standardized pattern:

## **Element-wise FFN**

In [1]:
examples off

In [2]:
# Some s-ops for basic information
bos = (tokens == "§");
quanti = ["∀", "∃"];  
LO = ["∧", "∨", "→"];
is_quanti = tokens in quanti;
is_LO = tokens in LO;

     s-op: bos
     list: quanti = ['∀', '∃']
     list: LO = ['∧', '∨', '→']
     s-op: is_quanti
     s-op: is_LO


In [3]:
# Select Quantifiers as keys
k_quanti = select(is_quanti, True, ==);
# Quantifiers as queries to attend other tokens
q_quanti = select(True, is_quanti, ==);

# Select Logical Operators as keys
k_LO = select(is_LO, True, ==);
# Logical Operators as queries to attend other tokens
q_LO = select(True, is_LO, ==);

# Select the <bos> token as the key
k_bos = select(bos, True, ==);
# The <bos> token as query to attend other tokens
q_bos = select(True, bos, ==);

     selector: k_quanti
     selector: q_quanti
     selector: k_LO
     selector: q_LO
     selector: k_bos
     selector: q_bos


In [4]:
# Boolean value for opening and close brackets
opens = (tokens == "(");
closes = (tokens == ")");

     s-op: opens
     s-op: closes


In [5]:
# Select the position based on indices
select_all = select(1, 1, ==);
leftward = select(indices, indices, <=);  
rightward = select(indices, indices, >=); 
equal = select(indices, indices, ==);

     selector: select_all
     selector: leftward
     selector: rightward
     selector: equal


## **Main Algorithms**

In [6]:
examples on

### **Standard-1 with BOS**: Simple FOL with no LO

In [1]:
# Select quanti from both keys and queries
def type1(){
    QD = aggregate(k_quanti and q_bos, indices);
    return QD[0];
}

     console function: type1()


#### Standard-1 with BOS: Test Cases (3/3)

In [14]:
type1_1 = "§∀uV(u)"; # QD=1
type1_2 = "§∃x∀yP(x,y)"; # QD=2
type1_3 = "§∀s∃b∃mX(sbm)"; # QD=3

    value: type1_1 =  "§∀uV(u)"
    value: type1_2 =  "§∃x∀yP(x,y)"
    value: type1_3 =  "§∀s∃b∃mX(sbm)"


In [11]:
set example type1_1
QD_type1_1 = type1();
# draw(type1(), "§∀uV(u)");

     s-op: QD_type1_1
 	 Example: QD_type1_1("§∀uV(u)") = [1]*7 (ints)


In [12]:
set example type1_2
QD_type1_2 = type1();
draw(type1(), "§∃x∀yP(x,y)");

     s-op: QD_type1_2
 	 Example: QD_type1_2("§∃x∀yP(x,y)") = [2.0]*11 (floats)
	 =  [2.0]*11 (floats)


In [15]:
set example type1_3
QD_type1_3 = type1();
# draw(type1(), "§∀s∃b∃mX(sbm)");

     s-op: QD_type1_3
 	 Example: QD_type1_3("§∀s∃b∃mX(sbm)") = [3.0]*13 (floats)
	 =  [3.0]*13 (floats)


### **Standard-2 with BOS**: Simple FOL with one LO
- It can also process **Standard-1 with BOS**.

In [15]:
def type2(){
    # Store the information in the <bos> token
    bos_quanti = k_quanti and q_bos;
    bos_LO = k_LO and q_bos;
    LO_idx = aggregate(bos_LO, indices);
    left_quanti = k_quanti and select(indices, LO_idx, <);
    right_quanti = bos_quanti and select(indices, LO_idx, >);
    left_QD = selector_width(left_quanti)[0];
    right_QD = selector_width(right_quanti)[0];
    QD = left_QD if (left_QD > right_QD) else right_QD;
    return QD;
}

     console function: type2()


#### Standard-2 with BOS: Test Cases (6/6)

In [16]:
type2_1 = "§∀tC(t)→∃iS(i)"; # QD=1
type2_2 = "§∃dF(d)→∀z∀tD(z,t)"; # QD=2
type2_3 = "§∃y∀lG(y,l)→∀k∃pD(k,p)"; # QD=2
type2_4 = "§∀x∀hR(h,x)∨∃f∀qW(q,f)"; # QD=2
type2_5 = "§∀xP(x)∨∀y∃z∀wQ(y,z,w)"; # QD=3
type2_6 = "§∀d∃g∀iY(d,g,i)∧∃t∀m∀bT(t,m,b)"; # QD=3

    value: type2_1 =  "§∀tC(t)→∃iS(i)"
    value: type2_2 =  "§∃dF(d)→∀z∀tD(z,t)"
    value: type2_3 =  "§∃y∀lG(y,l)→∀k∃pD(k,p)"
    value: type2_4 =  "§∀x∀hR(h,x)∨∃f∀qW(q,f)"
    value: type2_5 =  "§∀xP(x)∨∀y∃z∀wQ(y,z,w)"
    value: type2_6 =  "§∀d∃g∀iY(d,g,i)∧∃t∀m∀bT(t,m,b)"


In [18]:
set example type2_1
QD_type2_1 = type2();
# draw(type2(), "§∀tC(t)→∃iS(i)");

     s-op: QD_type2_1
 	 Example: QD_type2_1("§∀tC(t)→∃iS(i)") = [1]*14 (ints)


In [17]:
set example type2_2
QD_type2_2 = type2();
# draw(type2(), "§∃dF(d)→∀z∀tD(z,t)");

     s-op: QD_type2_2
 	 Example: QD_type2_2("§∃dF(d)→∀z∀tD(z,t)") = [2]*18 (ints)


In [18]:
set example type2_3
QD_type2_3 = type2();
# draw(type2(), "§∃y∀lG(y,l)→∀k∃pD(k,p)");

     s-op: QD_type2_3
 	 Example: QD_type2_3("§∃y∀lG(y,l)→∀k∃pD(k,p)") = [2]*22 (ints)


In [19]:
set example type2_4
QD_type2_4 = type2();
# draw(type2(), "§∀x∀hR(h,x)∨∃f∀qW(q,f)");

     s-op: QD_type2_4
 	 Example: QD_type2_4("§∀x∀hR(h,x)∨∃f∀qW(q,f)") = [2]*22 (ints)


In [20]:
set example type2_5
QD_type2_5 = type2();
# draw(type2(), "§∀xP(x)∨∀y∃z∀wQ(y,z,w)");

     s-op: QD_type2_5
 	 Example: QD_type2_5("§∀xP(x)∨∀y∃z∀wQ(y,z,w)") = [3]*22 (ints)


In [21]:
set example type2_6
QD_type2_6 = type2();
# draw(type2(), "§∀d∃g∀iY(d,g,i)∧∃t∀m∀bT(t,m,b)");

     s-op: QD_type2_6
 	 Example: QD_type2_6("§∀d∃g∀iY(d,g,i)∧∃t∀m∀bT(t,m,b)") = [3]*30 (ints)


### **Standard-3 with BOS**: Simple QD allowing unlimited LOs
- It can also process **Standard-2 with BOS**.

In [24]:
def type3(){
    LO_bos = k_bos and q_LO;
    LO_pos = aggregate(LO_bos, 1, 0);
    quanti_bos = k_bos and q_quanti;
    quanti_pos = aggregate(quanti_bos, 1, 0);
    
    clause_idx = aggregate(leftward, LO_pos)*(indices+1);
    same_clause = select(clause_idx, clause_idx, ==);
    freq = selector_width(same_clause);

    clause_quanti_pos = aggregate(same_clause, quanti_pos);
    clause_QD = clause_quanti_pos * freq;
    QD = sort(clause_QD, clause_QD)[-1];
    QD_type3 = aggregate(k_bos,QD);
    return QD_type3;
}

     console function: type3()


#### Standard-3 with BOS: Test Cases (7/7)

In [26]:
type3_1 = "§∀jE(j)∧∃qK(q)→∀rM(r)"; # QD=1
type3_2 = "§∃fL(f)∧∀a∀uW(au)∨∀sZ(s)"; # QD=2
type3_3 = "§∃xF(x)∧∃fD(f)→∀r∀tY(rt)"; # QD=2
type3_4 = "§∀cD(c)∧∃w∀dV(wd)→∃s∀tT(st)"; # QD=2
type3_5 = "§∀wD(w)∧∀u∃l∃jF(ulj)∧∃iO(i)"; # QD=3
type3_6 = "§∃vR(v)∧∀k∀t∃iN(kti)→∀d∀g∃bP(dgb)"; # QD=3
type3_7 ="§∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)"; # QD=2

    value: type3_1 =  "§∀jE(j)∧∃qK(q)→∀rM(r)"
    value: type3_2 =  "§∃fL(f)∧∀a∀uW(au)∨∀sZ(s)"
    value: type3_3 =  "§∃xF(x)∧∃fD(f)→∀r∀tY(rt)"
    value: type3_4 =  "§∀cD(c)∧∃w∀dV(wd)→∃s∀tT(st)"
    value: type3_5 =  "§∀wD(w)∧∀u∃l∃jF(ulj)∧∃iO(i)"
    value: type3_6 =  "§∃vR(v)∧∀k∀t∃iN(kti)→∀d∀g∃bP(dgb)"
    value: type3_7 =  "§∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)"


In [95]:
set example type3_1
QD_type3_1 = type3();
# draw(type3(), "§∀jE(j)∧∃qK(q)→∀rM(r)");

     s-op: QD_type3_1
 	 Example: QD_type3_1("§∀jE(j)∧∃qK(q)→∀rM(r)") = [1.0]*21 (floats)


In [96]:
set example type3_2
QD_type3_2 = type3();
# draw(type3(), "§∃fL(f)∧∀a∀uW(au)∨∀sZ(s)");

     s-op: QD_type3_2
 	 Example: QD_type3_2("§∃fL(f)∧∀a∀uW(au)∨∀sZ(s)") = [2.0]*24 (floats)


In [97]:
set example type3_3
QD_type3_3 = type3();
# draw(type3(), "§∃xF(x)∧∃fD(f)→∀r∀tY(rt)");

     s-op: QD_type3_3
 	 Example: QD_type3_3("§∃xF(x)∧∃fD(f)→∀r∀tY(rt)") = [2.0]*24 (floats)


In [98]:
set example type3_4
QD_type3_4 = type3();
# draw(type3(), "§∀cD(c)∧∃w∀dV(wd)→∃s∀tT(st)");

     s-op: QD_type3_4
 	 Example: QD_type3_4("§∀cD(c)∧∃w∀dV(wd)→∃s∀tT(st)") = [2.0]*27 (floats)


In [99]:
set example type3_5
QD_type3_5 = type3();
# draw(type3(), "§∀wD(w)∧∀u∃l∃jF(ulj)∧∃iO(i)");

     s-op: QD_type3_5
 	 Example: QD_type3_5("§∀wD(w)∧∀u∃l∃jF(ulj)∧∃iO(i)") = [3.0]*27 (floats)


In [100]:
set example type3_6
QD_type3_6 = type3();
# draw(type3(), "§∃vR(v)∧∀k∀t∃iN(kti)→∀d∀g∃bP(dgb)");

     s-op: QD_type3_6
 	 Example: QD_type3_6("§∃vR(v)∧∀k∀t∃iN(kti)→∀d∀g∃bP(dgb)") = [3.0]*33 (floats)


In [27]:
set example type3_7
QD_type3_7 = type3();
# draw(type3(), "§∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)");

     s-op: QD_type3_7
 	 Example: QD_type3_7("§∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)") = [2.0]*25 (floats)
	 =  [2.0]*25 (floats)
